# `Data parsing (Glassdoor)`

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('datascientist_salary_glassdoor.csv')

### Analysing the salary coloumn

- Make a col "Salary (Employer Est)" and input salaries using Salary coloumn else -1
- Make a col "Salary (Glassdoor Est) " and input salaries using salary coloumn else -1

In [2]:
df['Salary (Employer est.)'] = np.where(df['Salary'].str.endswith('(Employer Est.)'),'1','0')
df['Salary (Glassdoor est.)'] = np.where(df['Salary'].str.endswith('(Glassdoor Est.)'),'1','0')

# In this code snippet, df['Salary'] selects the 'Salary' column of the DataFrame. 
# The str.endswith() method checks if each entry in the column ends with "..some string". 
# If the condition is True, the entry is copied to the new Column. 
# Otherwise, '-1' is assigned to the new col for that entry. 

- Cleaning the salary col

In [4]:
df['Salary'] = df['Salary'].str.replace(' \(Employer Est.\)', '', regex=True)
df['Salary'] = df['Salary'].str.replace(' \(Glassdoor Est.\)', '', regex=True)

- Removing the currency signs

In [5]:
df['Salary'] = df['Salary'].str.replace('₹','',regex =True)

- Making sure the salaries are in the same units (we chose it to be Lacs)
- We may have data like T-T,T,T-L

In [7]:
# tackling T-T data type
df['Salary'] = df['Salary'].str.replace('(\d+)T - (\d+)T', lambda x: str(int(x.group(1))/100) + 'L - ' + str(int(x.group(2))/100) + 'L', regex=True)

# tackling T-L data type
df['Salary'] = df['Salary'].str.replace('(\d+)T - (\d+)L', lambda x: str(int(x.group(1))/100) + 'L - ' + str(int(x.group(2))) + 'L', regex=True)

# tackling T data type
df['Salary'] = df['Salary'].str.replace('(\d+)T', lambda x: str(int(x.group(1))/100) + 'L', regex=True)

- Converting hourly salary to annual

In [9]:
# tacking "per hour" type data
# first getting rid of the commas

df['Salary'] = df['Salary'].str.replace(',','',regex=True)

# converting the per hour salary into annual by assuming 2100 working hours.
hour_L = 2100/100000
df['Salary'] = df['Salary'].apply(lambda x: str(np.round(float(x.rstrip('Per hour').split('-')[0])*hour_L,2))+'L - '+ \
str(np.round(float(x.rstrip('Per hour').split('-')[1])*hour_L,2))+'L' if 'Per hour' in x else x)


- Remove denomination from the end and we will end up with a range

In [11]:
df['Salary'] = df['Salary'].str.replace('L','',regex =True)

- Renaming the salary col to mention the units of our data

In [12]:
df.rename(columns={'Salary': 'Salary (L)'}, inplace=True)

- Defining two cols for min and max salary

In [14]:
df['min Salary (L)'] = df['Salary (L)'].apply(lambda x: x.split('-')[0])
df['max Salary (L)'] = df['Salary (L)'].apply(lambda x: x.split('-')[1] if len(x.split('-')) > 1 else x.split('-')[0])

- Making a col with average salary

In [16]:
df['avg salary (L)'] = df['Salary (L)'].apply(lambda x: str((float(x.split('-')[0])+float(x.split('-')[1]))/2) if len(x.split('-')) > 1 else x.split('-')[0])


### Analysing 'company' coloumn

- The extracted company names are text only so it won't require further cleaning.

    While data scrapping we already removed the rating part using regex expression

    def text_clean(string):
        pattern = r"\d+(\.\d+)? ★$"  # Regex pattern to match the substring "3.9 ★" at the end of the string
        stripped_string = re.sub(pattern, "", string)  # Remove the matched substring using regex substitution
        return stripped_string


### Analysing 'Location' coloumn

- We use 'info()' method to find out that for 4 job listings, there is no location information
we drop these rows using the following command

In [27]:
df.dropna(subset=['Location'], inplace=True)

In [37]:
df.to_csv('salary_clean_glassdoor.csv',index=False)

In [18]:
print('*'*150)

******************************************************************************************************************************************************


# `Data Parsing (Naukri)`

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('datascientist_salary_naukri.csv')

- Dropping the rows for which salary is not disclosed

In [2]:
df = df[df['Salary']!='Not disclosed']
df = df.reset_index()
df = df.drop(['index'],axis=1)

In [3]:
# removing rows with salary in crores (may be potential outliers)
df.drop(index=[378,380],axis=0,inplace=True)
df = df.reset_index()
df = df.drop(['index'],axis=1)

- Expressing the salary in lacs

In [4]:
# removing commas between numbers

df['Salary'] = df['Salary'].str.replace(',','',regex=True)

# coversion to lacs
df['Salary (L)'] = df['Salary'].apply(lambda x: str(float(x.rstrip('Lacs PA').split('-')[0])/100000)+ \
                                      '-'+x.rstrip('Lacs PA').split('-')[1]+' Lacs PA' if (('Lacs PA' in x) and (len(x.rstrip('Lacs PA').split('-')[0])>4)) else x)

df['Salary (L)'] = df['Salary (L)'].apply(lambda x: str(float(x.rstrip('PA').split('-')[0])/100000) +\
                                           '-' + str(float(x.rstrip('PA').split('-')[1])/100000) + ' Lacs PA' if (('Lacs' not in x) and ('Cr' not in x) and (len(x.rstrip('PA').split('-')) > 1)) else x)


df['Salary (L)'] = df['Salary (L)'].apply(lambda x: str(float(x.rstrip('PA'))/100000) +\
                                         ' Lacs PA' if ('PA' in x and 'Lacs' not in x) else x)

- some more cleaning of jargons

In [5]:
pattern = r"\(Including Variable: \d+(\.\d+)?%\)"
df['Salary (L)'] = df['Salary (L)'].replace(pattern,'',regex=True)


In [6]:
df['Salary (L)'] = df['Salary (L)'].replace('Lacs PA','',regex=True)

- construct cols with min, max and average salary

In [7]:
df['min Salary (L)'] = df['Salary (L)'].apply(lambda x: x.split('-')[0])
df['max Salary (L)'] = df['Salary (L)'].apply(lambda x: x.split('-')[1] if len(x.split('-')) > 1 else x.split('-')[0])
df['avg salary (L)'] = df['Salary (L)'].apply(lambda x: str((float(x.split('-')[0])+float(x.split('-')[1]))/2) if len(x.split('-')) > 1 else x.split('-')[0])


### Analysing `experience` coloumn

In [11]:
df['Experience (yrs)'] = df['Experience'].str.replace('Yrs','',regex=True)

In [13]:
df['min Experience (yrs)'] = df['Experience (yrs)'].apply(lambda x: x.split('-')[0])
df['max Experience (yrs)'] = df['Experience (yrs)'].apply(lambda x: x.split('-')[1] if len(x.split('-')) > 1 else x.split('-')[0])
df['avg Experience (yrs)'] = df['Experience (yrs)'].apply(lambda x: str((float(x.split('-')[0])+float(x.split('-')[1]))/2) if len(x.split('-')) > 1 else x.split('-')[0])


- Saving the cleaned dataset

In [15]:
df.to_csv('salary_clean_naukri.csv',index=False)